In [1]:
import json
import pandas as pd
from soccerapi.api import Api888Sport

In [2]:
api = Api888Sport()
odds = api.competitions()

print(odds)

{'Champions League': {'Champions League': 'https://www.888sport.com/#/filter/football/champions_league/'}, 'Europa League': {'Europa League': 'https://www.888sport.com/#/filter/football/europa_league/'}, 'Italy': {'Serie A': 'https://www.888sport.com/#/filter/football/italy/serie_a/', 'Coppa Italia': 'https://www.888sport.com/#/filter/football/italy/coppa_italia/', 'Serie C Girone B': 'https://www.888sport.com/#/filter/football/italy/serie_c_girone_b/'}, 'England': {'Premier League': 'https://www.888sport.com/#/filter/football/england/premier_league/', 'The Championship': 'https://www.888sport.com/#/filter/football/england/the_championship/', 'FA Cup': 'https://www.888sport.com/#/filter/football/england/fa_cup/', 'League One': 'https://www.888sport.com/#/filter/football/england/league_one/', 'League Two': 'https://www.888sport.com/#/filter/football/england/league_two/', 'EFL Cup': 'https://www.888sport.com/#/filter/football/england/efl_cup/', 'Super League (W)': 'https://www.888sport.c

In [3]:
# url = 'https://www.888sport.com/#/filter/football/italy/serie_a'
url = 'https://www.888sport.com/#/filter/football/england/premier_league/'
odds = api.odds(url)

# print(odds)
print(json.dumps(odds, sort_keys=True, indent=4))

[
    {
        "away_team": "Watford",
        "both_teams_to_score": {
            "no": 1900,
            "yes": 1870
        },
        "double_chance": {
            "12": 1190,
            "1X": 1090,
            "2X": 3050
        },
        "full_time_result": {
            "1": 1370,
            "2": 7500,
            "X": 4900
        },
        "home_team": "West Ham",
        "time": "2022-02-08T19:45:00Z",
        "under_over": {
            "O2.5": 1640,
            "U2.5": 2250
        }
    },
    {
        "away_team": "Everton",
        "both_teams_to_score": {
            "no": 2120,
            "yes": 1700
        },
        "double_chance": {
            "12": 1330,
            "1X": 1480,
            "2X": 1520
        },
        "full_time_result": {
            "1": 2550,
            "2": 2700,
            "X": 3350
        },
        "home_team": "Newcastle",
        "time": "2022-02-08T19:45:00Z",
        "under_over": {
            "O2.5": 1910,
            "

In [4]:
odds_df_ = pd.json_normalize(odds)
odds_df = odds_df_[['time', 'home_team', 'away_team', 'full_time_result.1', 'full_time_result.X', 'full_time_result.2']]

odds_df

,time,home_team,away_team,full_time_result.1,full_time_result.X,full_time_result.2
0,2022-02-08T19:45:00Z,West Ham,Watford,1370,4900,7500
1,2022-02-08T19:45:00Z,Newcastle,Everton,2550,3350,2700
2,2022-02-08T20:00:00Z,Burnley,Manchester United,6100,4000,1530
3,2022-02-09T19:45:00Z,Norwich City,Crystal Palace,3450,3300,2140
4,2022-02-09T19:45:00Z,Tottenham,Southampton,1620,4000,5100
5,2022-02-09T19:45:00Z,Manchester City,Brentford,1090,9500,25000
6,2022-02-09T20:00:00Z,Aston Villa,Leeds United,1780,3900,4100
7,2022-02-10T19:45:00Z,Liverpool,Leicester City,1270,6100,9000
8,2022-02-10T19:45:00Z,Wolves,Arsenal,3350,3250,2180
9,2022-02-12T12:30:00Z,Manchester United,Southampton,1540,4300,5500


In [67]:
# Code for the Odds-Bias strategy as recommended in Chapter 12 of Soccermatics by David Sumpter

for index, row, in odds_df.iterrows():
    print("Match", index+1)
    print(row["home_team"], "vs", row["away_team"], "\n")
    
    o_home_win = 10000/row["full_time_result.1"]
    o_draw = 10000/row["full_time_result.X"]
    o_away_win = 10000/row["full_time_result.2"]
    print("Odds for a", row["home_team"], "win =", o_home_win)
    print("Odds for a draw =", o_draw)
    print("Odds for a", row["away_team"], "win =", o_away_win, "\n")
    
    o_total = (o_home_win + o_draw + o_away_win)
    o_home_corrected = o_home_win / o_total
    o_draw_corrected = o_draw / o_total
    o_away_corrected = o_away_win / o_total
    print("Corrected odds for a", row["home_team"], "win =", o_home_corrected)
    print("Corrected odds for a draw =", o_draw_corrected)
    print("Corrected odds for a", row["away_team"], "win =", o_away_corrected)
    
    if (abs(o_home_corrected - o_away_corrected) > 0.4):
        print("\nChecking for a possible bet on a win...............")
        p_home_win = 1/(1+(0.961*(pow(o_home_corrected/(1-o_home_corrected),-1.06))))
        p_away_win = 1/(1+(0.961*(pow(o_away_corrected/(1-o_away_corrected),-1.06))))
    
        if (p_home_win > p_away_win):
            print(row["home_team"], "playing at home appear to be the favourites!")
            print("p_home_win =", p_home_win, "| 1/o_home_win", (1/o_home_win))
            if (p_home_win > (1/o_home_win)):
                print("Check completed!")
                print("Place a bet on a", row["home_team"], "win!!!")
                print("\n==========================================================================\n")                
                continue
        else:
            print(row["away_team"], "playing away appear to be the favourites!")
            print("p_away_win =", p_away_win, "| 1/o_away_win", (1/o_away_win))
            if (p_away_win > (1/o_away_win)):
                print("Check completed!")
                print("Place a bet on a", row["away_team"], "win!!!")
                print("\n==========================================================================\n")                
                continue
        print("Check completed!")
        print("No bet recommended!")
        print("\n==========================================================================\n")                
        continue
    elif (abs(o_home_corrected - o_away_corrected) < 0.15):
        print("\nChecking for a possible bet on a draw...............")
        print("Both teams appear to be evenly matched!")
        p_draw = (0.355-(0.25*(abs(o_home_corrected - o_away_corrected))))
        print("p_draw =", p_draw, "| 1/o_draw =", 1/o_draw)
        if (p_draw > (1/o_draw)):
            print("Check completed!")
            print("Place a bet on a draw!!!")
            print("\n==========================================================================\n")                
            continue
        else:
            print("Check completed!")
            print("No bet recommended!")
            print("\n==========================================================================\n")                
            continue
            
    print("\nNo checks necessary!")
    print("No bet recommended!")
    print("\n==========================================================================\n")    

Match 1
West Ham vs Watford 

Odds for a West Ham win = 7.299270072992701
Odds for a draw = 2.0408163265306123
Odds for a Watford win = 1.3333333333333333 

Corrected odds for a West Ham win = 0.6838736089917749
Corrected odds for a draw = 0.1912054784323942
Corrected odds for a Watford win = 0.12492091257583086

Checking for a possible bet on a win...............
West Ham playing at home appear to be the favourites!
p_home_win = 0.7021819322448964 | 1/o_home_win 0.137
Check completed!
Place a bet on a West Ham win!!!


Match 2
Newcastle vs Everton 

Odds for a Newcastle win = 3.9215686274509802
Odds for a draw = 2.985074626865672
Odds for a Everton win = 3.7037037037037037 

Corrected odds for a Newcastle win = 0.36959852896107875
Corrected odds for a draw = 0.2813361936867913
Corrected odds for a Everton win = 0.34906527735212994

Checking for a possible bet on a draw...............
Both teams appear to be evenly matched!
p_draw = 0.3498666870977628 | 1/o_draw = 0.33499999999999996
C